In [31]:
# pip install -r dependencies.txt

In [32]:
import pandas as pd
import numpy as np
from scipy.optimize import linear_sum_assignment
import time
import secrets
# import matplotlib.pyplot as plt
# import networkx as nx

In [33]:
# Carregar dados CSV
diplomatas = pd.read_csv('../resources/aed_alocacao_recursos - diplomatas.csv')
cidades = pd.read_csv('../resources/aed_alocacao_recursos - cidades.csv')

# print(diplomatas.info())
# print(cidades.info())

In [34]:
def get_posto_trabalho(id, cidades):
  return cidades.loc[cidades['posto'] == id].to_dict(orient='records')[0]

get_posto_trabalho('P1', cidades)

{'cidade': 'C1', 'posto': 'P1', 'classificacao': 'A'}

In [35]:
def calcular_custo_aresta(diplomata, posto, cidades, atenuador = False):
    """
    Calcula o custo da aresta x diplomata-posto

    :param diplomata: Aresta produtora que representa o diplomata (pandas Series)
    :param posto: Aresta consumidora que representa a posto de trabalho (pandas Series)
    :param cidades: Conjunto de cidades com seus respectivos postos (pandas DataFrame)
    :param atenuador: Indica a necessidade de atribuir menor custo aos postos de menor classificação (boolean)
    :return: custo (float)
    """
    seed = secrets.randbits(128)
    CUSTO_INFINITO = 999999999.99
    
    # print(f"Calculando custo da aresta {diplomata.id} -> {posto.posto}")
    # Se tiver 12 anos fora, deve retornar
    if diplomata.tempo_fora >= 144:
        if posto.classificacao == '*':
            return 0.0
        else:
            return CUSTO_INFINITO
    # Se tiver 2 anos na lotação atual, deve trocar de lotacao
    if diplomata.tempo_na_lotacao >= 24 and diplomata.lotacao == posto.posto:
        return CUSTO_INFINITO

    # verificar se a origem á compatível com o destino. Ex. a só pode ir para B, C ou D
    if diplomata.lotacao != 'P0':
        posto_atual = get_posto_trabalho(diplomata.lotacao, cidades)
        # print('Posto atual {}'.format(posto_atual))
        if diplomata.lotacao != posto.posto and posto.classificacao == 'A':
            if posto_atual.get('classificacao')  == 'A':
                return CUSTO_INFINITO
    elif diplomata.pedagio < 24 and posto.classificacao == 'A':
        # Se estiver a menos de 2 anos no Brasil, não pode ir para um posto 'A'
        return CUSTO_INFINITO

    if posto.classificacao == 'A':
        return 1.0 

    if posto.classificacao == 'B':
        return 2.0

    if posto.classificacao == 'C':
        return 3.0 if not atenuador else 3.0 * np.random.default_rng(seed).random()

    if posto.classificacao == 'D':
        return 4.0 if not atenuador else 4.0 * np.random.default_rng(seed).random()
    
    return CUSTO_INFINITO

In [36]:
def alocar_diplomatas(matriz_custos):
  """
  Resolve o problema de alocação usando o algoritmo húngaro.

  :param matriz_custos: Matriz de custos (2D list ou numpy array)
  :return: Tupla (lista de alocações, custo total)
  """
  # Usando a função `linear_sum_assignment` da biblioteca scipy
  matriz_custos = np.array(matriz_custos)
  linhas, colunas = linear_sum_assignment(matriz_custos)
  custo_total = matriz_custos[linhas, colunas].sum()

  return linhas, colunas, custo_total

In [37]:
def calcular_custos_arestas(diplomatas, cidades, atenuador = False):
    diplomatas_tratado = diplomatas.sort_values("tempo_servico", ascending = False)
    pesos_arestas = []
    for d, row in diplomatas_tratado.iterrows():
        pesos_diplomata = []
        for i, posto in cidades.iterrows():
            pesos_diplomata.append(calcular_custo_aresta(row, posto, cidades, atenuador))
        pesos_arestas.append(pesos_diplomata)

    return pesos_arestas

In [38]:
# Exemplo de uso
# pesos_arestas = calcular_custos_arestas(diplomatas,cidades)
# linhas, colunas, custo = alocar_diplomatas(pesos_arestas)
# diplomatas_tratado = diplomatas.sort_values("tempo_servico", ascending = False)
# print("Alocações (diplomata -> posto):")
# for i, j in zip(linhas, colunas):
#   print(f"Diplomata {diplomatas_tratado.iloc[i].id} -> Posto {cidades.iloc[j].posto}")

# print("Custo total:", custo)

In [39]:
# np.random.randint(1,2)
a = [1,2]
len(a)

2

In [40]:
def gerar_base_diplomatas(qtd_diplomatas, qtd_postos):
  """
  Gera uma base de diplomatas e suas respectivas lotações com dados sintéticos

  :param qtd_diplomatas: Quantidade de diplomatas da base de testes (int)
  :param qtd_postos: Quantidade de postos únicos existentes. Deve ser maior ou igual à quantidade de diplomatas.
  :return: DataFrame
  """
  if qtd_diplomatas > qtd_postos:
    raise ValueError('A quantidade de diplomatas deve ser maior ou igual à quantidade de postos únicos.')

  # Geração da lotação
  ids = []
  pedagios = []
  lotacoes = []
  tempos_servico = []
  tempos_lotacao = []
  tempos_fora = []

  for p in range(qtd_diplomatas):
    while True:
      lotacao = 'P' + str(np.random.randint(1,qtd_postos+1))
      if lotacao == 'P0' or lotacao not in lotacoes:
        break

    tempo_servico = np.random.randint(300)  # tempo de serviço em meses
    pedagio = 0                             # tempo no Brasil em meses
    tempo_na_lotacao = 0                    # tempo na lotação em meses
    tempo_fora = 0                          # tempo ininterrupro fora do Brasil em meses
    if lotacao == 'P0': # Ele está no Brasil
      pedagio = np.random.randint(30)
      tempo_na_lotacao = 0
      tempo_fora = 0
    else:
      pedagio = 0
      tempo_na_lotacao = np.random.randint(25)
      tempo_fora = tempo_na_lotacao + np.random.randint(97,122)

    ids.append('D' + str(p+1))
    pedagios.append(pedagio)
    lotacoes.append(lotacao)
    tempos_servico.append(tempo_servico)
    tempos_lotacao.append(tempo_na_lotacao)
    tempos_fora.append(tempo_fora)

  d = {
    'id': ids,
    'pedagio': pedagios,
    'lotacao': lotacoes,
    'tempo_servico': tempos_servico,
    'tempo_na_lotacao': tempos_lotacao,
    'tempo_fora': tempos_fora
  }
  return pd.DataFrame(data=d, index=[*range(qtd_diplomatas)])

In [41]:
# dpls = gerar_base_diplomatas(150,175)

# dpls.loc[(dpls['lotacao'] != 'P0') & (dpls['pedagio'] != 0)]

In [42]:
def gerar_base_postos(qtd_postos):
  postos = ['P0']
  classificacoes = ['*']
  tipos_classificacoes = ['A','B','C','D']

  for p in range(qtd_postos):
    posto = 'P' + str(p+1)
    postos.append(posto)
    classificacoes.append(tipos_classificacoes[np.random.randint(4)])

  d = {
    'posto': postos,
    'classificacao': classificacoes
  }

  return pd.DataFrame(data=d, index=[*range(qtd_postos+1)])

In [43]:
def rodar_teste(qtd_diplomatas, qtd_postos, atenuador = False):
  # Gerar base de testes
  t_postos = gerar_base_postos(int(qtd_postos))
  t_diplomatas = gerar_base_diplomatas(qtd_diplomatas, qtd_postos)

  # Executar alocações
  inicio = time.time()
  t_pesos_arestas = calcular_custos_arestas(t_diplomatas, t_postos, atenuador)
  linhas, colunas, custo = alocar_diplomatas(t_pesos_arestas)
  fim = time.time()
  #preparar dados de retorno
  # pedagio,lotacao,tempo_servico,tempo_na_lotacao,tempo_fora
  t_diplomatas_tratado = t_diplomatas.sort_values("tempo_servico", ascending = False)
  t_diplomatas_tratado = t_diplomatas_tratado.merge(t_postos, how='inner', left_on='lotacao',right_on='posto')
  t_diplomatas_alocados = []
  t_postos_alocados = []
  t_classificacoes_atuais = []
  t_classificacoes_alocadas = []
  t_tempos_servicos = []
  t_pedagios = []
  t_lotacoes_anteriores = []
  t_tempos_nas_lotacoes = []
  t_tempos_fora = []

  for i, j in zip(linhas, colunas):
    diplomata = t_diplomatas_tratado.iloc[i].id
    tempo_servico = t_diplomatas_tratado.iloc[i].tempo_servico
    tempo_lotacao = t_diplomatas_tratado.iloc[i].tempo_na_lotacao
    tempo_fora = t_diplomatas_tratado.iloc[i].tempo_fora
    pedagio = t_diplomatas_tratado.iloc[i].pedagio
    lotacao = t_diplomatas_tratado.iloc[i].lotacao
    alocacao = t_postos.iloc[j].posto
    classificacao_atual = t_diplomatas_tratado.iloc[i].classificacao
    classificacao_alocada = t_postos.iloc[j].classificacao
    '''
    print(f"""
      'diplomata': {diplomata},
      'tempo_servico': {tempo_servico},
      'tempo_na_lotacao': {tempo_lotacao},
      'tempo_fora': {tempo_fora},
      'tempo_sede': {pedagio},
      'lotacao_anterior': {lotacao},
      'classificacao_anterior': {classificacao_atual},
      'lotacao_alocada': {alocacao},
      'classificacao_alocada': {classificacao_alocada}
          """)
    '''
    t_diplomatas_alocados.append(diplomata)
    t_tempos_servicos.append(tempo_servico)
    t_tempos_nas_lotacoes.append(tempo_lotacao)
    t_tempos_fora.append(tempo_fora)
    t_pedagios.append(pedagio)
    t_lotacoes_anteriores.append(lotacao)
    t_postos_alocados.append(alocacao)
    t_classificacoes_atuais.append(classificacao_atual)
    t_classificacoes_alocadas.append(classificacao_alocada)

  d= {
    'diplomata': t_diplomatas_alocados,
    'tempo_servico': t_tempos_servicos,
    'tempo_na_lotacao': t_tempos_nas_lotacoes,
    'tempo_fora': t_tempos_fora,
    'tempo_sede': t_pedagios,
    'lotacao_anterior': t_lotacoes_anteriores,
    'classificacao_anterior': t_classificacoes_atuais,
    'lotacao_alocada': t_postos_alocados,
    'classificacao_alocada': t_classificacoes_alocadas
  }
  df = pd.DataFrame(data=d, index=[*range(qtd_diplomatas)])
  tempo = fim - inicio
  # print("Custo total:", custo)
  # print(f"Tempo de execução igual a {fim - inicio}")
  return df, custo, tempo

In [44]:
[*range(5)]

[0, 1, 2, 3, 4]

In [ ]:
# df, custo, tempo =  rodar_teste(150)
qtd_diplomatas = 350
qtd_un_postos = 500
atenuador = False
df, custo, tempo =  rodar_teste(qtd_diplomatas, qtd_un_postos, atenuador)
df.to_csv('../output/resultados_n_' + str(qtd_diplomatas) + '.csv', index=False)
print(f"Custo total: {custo}")
print(f"Tempo de execução: {tempo}")

Custo total: 698.0
Tempo de execução: 36.61278820037842


In [48]:
# df, custo, tempo =  rodar_teste(150)
qtd_diplomatas = 350
qtd_un_postos = 500
atenuador = True
df, custo, tempo =  rodar_teste(qtd_diplomatas, qtd_un_postos, atenuador)
df.to_csv('../output/resultados_atenuados_n_' + str(qtd_diplomatas) + '.csv', index=False)
print(f"Custo total: {custo}")
print(f"Tempo de execução: {tempo}")

Custo total: 2000000093.1016202
Tempo de execução: 39.69610357284546


In [46]:
# média de tempo de serviço
# float(dplm['tempo_servico'].mean())/12

# Mediana do tempo de serviço
# float(dplm['tempo_servico'].median())/12

# dplm[['tempo_servico', 'tempo_na_lotacao', 'tempo_fora', 'pedagio']].describe()
# dplm[["posto","tempo_servico"]].groupby()

#####################
#### Calular a média de tempo de serviço por classe de posto antes e depois
####
####################
# print(df[['classificacao_anterior', 'tempo_servico','tempo_na_lotacao','tempo_fora','tempo_sede']].groupby('classificacao_anterior').mean())
# print(df[['classificacao_alocada', 'tempo_servico','tempo_na_lotacao','tempo_fora','tempo_sede']].groupby('classificacao_alocada').mean())

# df.loc[df['tempo_fora']>=144]

In [47]:
# Rodar testes em massa

# resultados = pd.DataFrame()
# for q in range(50, 550, 100):
#   df, custo, tempo =  rodar_teste(q)
#   df.to_csv('../output/resultados_n_'+str(q)+'.csv', index=False)